# Fall detector
## \[5ARB0\] Data Acquisition and Analysis - Technical Assignment

In [2]:
# this cell imports the libraries or packages that you can use during this assignment
# you are not allowed to import additional libraries or packages
from helpers import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import multivariate_normal

> **Important**
>
> Do not import any other packages or libraries than the ones already provided to you.
>
> Write your code between the `BEGIN_TODO` and `END_TODO` markers. Do not change these markers.
>
> Restart your notebook and run all cells before submission.

## Introduction
In this assignment you will use the previously learned data analysis methods and apply them to create a fall detector. After collecting and preprocessing sensor recording, you will extract a set of features which you will use for creating a fall detector.

This assignment is split into 4 parts. Parts 1-3 encompass the collection, preprocessing and feature extraction of sensor recordings. Finally, in part 4 you will create a fall detector.

### Learning goals
After this assignment you can
- collect data according to a protocol;
- load and merge datasets;
- preprocess data;
- extract features from data;
- apply analysis methods for creating a fall detector.

## Part 1: Data collection

Collect the data according to the data collection protocol provided on Canvas.

> (For iPhone) If you have issues accessing the (live) calibrated data of the _Sensor Logger_ app, you need to enable motion and compas calibration on your phone as explained [here](https://www.lifewire.com/how-to-calibrate-an-iphone-4172146). You will notice this once the live view and the export yield empty results.

## Part 2: Data cleaning and preprocessing
In this part you will be working on cleaning and preprocessing the data that you have gathered for further analysis.


---
---
### Exercise 2.1: Read, merge and select data
Load one of your individuals recordings in a Pandas DataFrame called `data`. You may want to have a look at the `pd.merge_asof` function to combine the recordings of the different sensors. Make sure that the `data` dataframe does not contain any `NaN`'s or empty fields as a result of different sampling frequencies. Any columns/recordings that you will not be using in your experiment should be removed from `data` (except for the `seconds_elapsed` column). Also remove duplicate colums. In the end your dataframe should have an indexing column, a column called `seconds_elapsed`, followed by the columns corresponding to the measurements.

In [2]:
#// BEGIN_TODO [5ARB0_FallDetector_2_1] Read, merge and select data

FolderPath = "RawData"
FolderOfIntrest = "ErikTrialData2023-09-22_14-33-27"
files = ["Accelerometer.csv","Gravity.csv","Gyroscope.csv"]
i=0
for file in files:
    path = FolderPath + "/" + FolderOfIntrest + "/" + file
    RawData[i] = pd.read_csv("RawData/ErikTrialData2023-09-22_14-33-27/Gravity.csv")
    i = i + 1

#// END_TODO [5ARB0_FallDetector_2_1]

In [28]:
acc = "./RawDataDaniel/Walk_samples/2023-09-26_07-15-41/Accelerometer.csv"
grav = "./RawDataDaniel/Walk_samples/2023-09-26_07-15-41/Gravity.csv"
gyro = "./RawDataDaniel/Walk_samples/2023-09-26_07-15-41/Gyroscope.csv"

acc_df = pd.read_csv(acc)
grav_df = pd.read_csv(grav)
gyro_df = pd.read_csv(gyro)

df = pd.merge_asof(acc_df, gyro_df, on='time', suffixes=['None', '_gyro'])
df = pd.merge_asof(df, grav_df, on='time', suffixes=['_acc', '_gyro'])

# df.drop(columns=['seconds_elapsed_gyro', ])
df


,time,seconds_elapsed_acc,z_acc,y_acc,x_acc,seconds_elapsed_gyro,z_gyro,y_gyro,x_gyro,seconds_elapsed,z,y,x
0,1695712542020082700,0.146083,0.000000,0.000000,0.000000,0.146083,0.278034,-0.664458,0.136221,0.146083,8.006465,5.622877,-0.670951
1,1695712542030123800,0.156124,-0.028530,0.103019,0.138830,0.156124,0.171507,-0.626109,0.304399,0.156124,7.946890,5.725518,-0.485553
2,1695712542040343800,0.166344,-0.240870,0.079999,-0.072005,0.166344,0.055394,-0.515321,0.441818,0.166344,7.867237,5.835296,-0.475711
3,1695712542050217000,0.176217,-0.523833,-0.059002,-0.245359,0.176217,-0.025566,-0.367250,0.507864,0.176217,7.787525,5.936728,-0.529233
4,1695712542060258000,0.186258,-0.646675,-0.103556,-0.354661,0.186258,-0.084023,-0.139416,0.491885,0.186258,7.710637,6.028436,-0.611899
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2290,1695712568785646300,26.911646,-0.165359,0.083551,0.158941,26.871540,0.036219,0.075767,-0.089615,26.911646,8.845161,4.194827,-0.580462
2291,1695712568795676700,26.921677,-0.204567,0.105250,0.154618,26.871540,0.036219,0.075767,-0.089615,26.921677,8.843606,4.198024,-0.581053
2292,1695712568805735400,26.931735,-0.093494,0.095814,0.145486,26.871540,0.036219,0.075767,-0.089615,26.931735,8.843465,4.198252,-0.581532
2293,1695712568815768600,26.941769,0.034523,0.081493,0.077031,26.871540,0.036219,0.075767,-0.089615,26.941769,8.843783,4.197780,-0.580108


In [ ]:
data

### End of exercise 2.1
---
---

In order to restrict our focus to fall detection, we would like to trim the recorded segment. In this way we can remove the movements corresponding to starting and stopping the sensor logger app. 

---
---
### Exercise 2.2: Trim data
Remove the first and last 5 seconds of the recordings for this purpose and save this trimmed data frame to `data_trimmed`. Make sure that your code works for a data frame containing an arbitrary number of columns.

In [ ]:
#// BEGIN_TODO [5ARB0_FallDetector_2_2] Trim data

# ===== =====> Replace this line by your code. <===== ===== #

#// END_TODO [5ARB0_FallDetector_2_2]

In [ ]:
data_trimmed

### End of exercise 2.2
---
---

---
---
### Exercise 2.3: Normalize recordings
For improved processing, the recordings should be normalized. Normalize the recordings by subtracting its mean and by then dividing by its standard deviation. Perform this normalization for each column individually. Save your normalized data in the the data frame `data_norm`.

In [ ]:
#// BEGIN_TODO [5ARB0_FallDetector_2_3] Normalize recordings

# ===== =====> Replace this line by your code. <===== ===== #

#// END_TODO [5ARB0_FallDetector_2_3]

In [ ]:
data_norm

### End of exercise 2.3
---
---

In [ ]:
# plot recordings
ex2_plot_data(data_norm)

## Part 3: Feature extraction
The current data is not yet suited for detecting a fall. Based on the measurements at a specific point of time, it is difficult to determine whether someone has allen. Instead, it would be more appropriate to perform the fall detection over _segments_ of time. In this part you will extract features that will be used for detecting falls. But first all collected datasets will be processed.

---
---
### Exercise 3.1: Merge all datasets
Before starting the feature extraction, merge all the preprocessed datasets obtained in the protocol. You will need to load all recording, and again perform all preprocessing steps of Part 2 for the individual recordings. Make sure your code adheres to proper coding standards (make it look nice, don't copy part 2 15 times). Save your merged data in the the data frame `data_merged`.

In [ ]:
#// BEGIN_TODO [5ARB0_FallDetector_3_1] Merge all datasets

# ===== =====> Replace this line by your code. <===== ===== #

#// END_TODO [5ARB0_FallDetector_3_1]

### End of exercise 3.1
---
---

---
---
### Exercise 3.2: Convert data frame to matrix
In order to extract features from the recording, first convert the dataframe to a Numpy matrix called `mat`. This matrix should have dimensions (_nr of time points_, _nr or different recordings_). Make sure that you remove the _seconds_elapsed_ column, as this does not yield any useful information for the fall detector.

In [ ]:
#// BEGIN_TODO [5ARB0_FallDetector_3_2] Convert data frame to matrix

# ===== =====> Replace this line by your code. <===== ===== #

#// END_TODO [5ARB0_FallDetector_3_2]

### End of exercise 3.2
---
---

Now that the data frame has been converted to a matrix, it can be split into different overlapping segments, of which we can extract features. As a starting point we will specify features as the mean value of a segment, its standard deviation, its minimum and its maximum.

---
---
### Exercise 3.3: Processing segments
Create a function `Y = process_segment(mat, fun, L, K)` that processes time segments of the matrix `mat`. The argument `fun` specifies the operation to be performed on the segment and its value comes from the set `["mean", "std", "minimum", "maximum"]`. `L` specifies the segment length and `K` specifies the number of samples overlap between segments. The function should return a matrix `Y` with dimensions (_nr of segments_, _nr of different recordings_)

In [ ]:
#// BEGIN_TODO [5ARB0_FallDetector_3_3] Process segments

# ===== =====> Replace this line by your code. <===== ===== #

#// END_TODO [5ARB0_FallDetector_3_3]

In [ ]:
Y_mean = process_segment(mat, "mean", 100, 50)
Y_std = process_segment(mat, "std", 100, 50)
Y_minimum = process_segment(mat, "minimum", 100, 50)
Y_maximum = process_segment(mat, "maximum", 100, 50)

### End of exercise 3.3
---
---

---
---
### Exercise 3.4: Concatenate features
Now that you have computed some features of the recordings, it becomes necessary to combine them into a single matrix. Create the matrix `features` which concatenates the above results along the appropriate axis.

In [ ]:
#// BEGIN_TODO [5ARB0_FallDetector_3_4] Concatenate features

# ===== =====> Replace this line by your code. <===== ===== #

#// END_TODO [5ARB0_FallDetector_3_4]

### End of exercise 3.4
---
---

## Part 4: Fall detector
In this part of the assignment we will use the previously implemented data analysis methods to create a simple fall detector. You will be given more freedom to experiment with the different techniques used.

During this part you can make use of the `sklearn` (Scikit learn) package. This package offers some benefits over the handwritten clustering functions. These algorithms are numerically stable and better optimized to run on large data sets.

In [ ]:
from sklearn.decomposition import PCA, FastICA
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

---
---
### Exercise 4.1: Fall detector
In this assignment you will create your own fall detector, using the previously recorded data and learned data analysis methods. You are free to add more features to the dataset if you want. Possible steps include: 1) feature extraction, 2) data compression and 3) clustering.

In [ ]:
#// BEGIN_TODO [5ARB0_FallDetector_4_1] Fall detector

# ===== =====> Replace this line by your code. <===== ===== #

#// END_TODO [5ARB0_FallDetector_4_1]

### End of exercise 4.1
---
---

---
---
### Exercise 4.2: Fall classification
Use your fall detector to classify when someone has fallen. Plot your fall classification over time.

In [ ]:
#// BEGIN_TODO [5ARB0_FallDetector_4_2] Classify falls

# ===== =====> Replace this line by your code. <===== ===== #

#// END_TODO [5ARB0_FallDetector_4_2]

### End of exercise 4.2
---
---

---
---
### Exercise 4.3: Classification improvements
You might have noticed that it is not easy to create a fall detector. How do you think that you could improve the performance? Please carefully describe the current flaws and ways to deal with them, together with a list of next steps that you would take. Please elaborate on your answer.

### End of exercise 4.3
---
---

>   Make sure to restart this notebook and to rerun all cells before submission to check whether all code runs properly.